pip install seaborn

pip install numpy==1.23.4

In [3]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
print(tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as C
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model,Model
from tensorflow.python.keras.utils import conv_utils
from tensorflow.keras import regularizers, constraints, initializers
import pandas as pd
from tensorflow.keras.layers import GlobalAveragePooling2D
# import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

2.6.0


In [4]:
train_path=('/LAB/Brain Tumor/Brain Tumor MRI/train')
test_path=('/LAB/Brain Tumor/Brain Tumor MRI/test')
val_path=('/LAB/Brain Tumor/Brain Tumor MRI/val')
batch_size = 32
img_height = 512
img_width = 512
no_of_clases = 4
clsses_name = ['glioma','meninginoma','notumor','pituitary tumor']
input_shape = (img_height,img_width,1)

datagen = ImageDataGenerator(rescale = 1./255,featurewise_center=True,horizontal_flip=False,vertical_flip = False,validation_split = 0.1,featurewise_std_normalization = True)

train_generator = datagen.flow_from_directory(train_path,target_size=(img_height,img_width),batch_size = batch_size, shuffle =True,class_mode = 'categorical')

validation_generator = datagen.flow_from_directory(val_path,target_size=(img_height,img_width),batch_size = batch_size, shuffle =True,class_mode = 'categorical')

print('train classes')
print(train_generator.class_indices)
print('train labels')
print(train_generator.labels)
print('validation clases')
print(validation_generator.class_indices)
print('validation labels')
print(validation_generator.labels)

Found 5617 images belonging to 4 classes.
Found 701 images belonging to 4 classes.
train classes
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
train labels
[0 0 0 ... 3 3 3]
validation clases
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
validation labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

In [5]:
print(len(train_generator))

176


In [ ]:
with tf.device('/GPU:1'):
    pretrained_model = tf.keras.applications.InceptionResNetV2(
                    input_shape=(512, 512, 3),
                    include_top=False,
                    weights='imagenet',
                    pooling='avg',
                    )

pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input
outputs = tf.keras.layers.Dense(4, activation='softmax')(pretrained_model.output)

model = tf.keras.Model(inputs, outputs)
print(model.summary())

In [ ]:
optimizers = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer=optimizers,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
with tf.device('/GPU:1'):
    history = model.fit(train_generator,validation_data=validation_generator,epochs=25)

In [ ]:
plt.plot(history.history['loss'],label='Train_loss')
plt.plot(history.history['val_loss'],label='Val_loss')
plt.legend()
plt.xlabel('No of Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.show()


plt.plot(history.history['accuracy'],label = 'Train_acc')
plt.plot(history.history['val_accuracy'],label = 'Val_acc')
plt.legend()
plt.xlabel('No. of Epochs')
plt.ylabel("Accuracy")
plt.title("Accuracy vs Epochs")
plt.show()

In [ ]:
test_data = ImageDataGenerator(rescale=1./255)
test_generator = test_data.flow_from_directory(test_path, target_size=(512,512), batch_size=32, shuffle=False, class_mode="categorical")

In [ ]:
predictions = np.argmax(model.predict(test_generator), axis=1)
matrix = confusion_matrix(test_generator.labels, predictions)
report= classification_report(test_generator.labels, predictions, target_names=test_generator.class_indices, zero_division=0)

In [ ]:
pred = model.predict(test_generator,verbose=1)
test_generator_indices = np.argmax(pred,axis=1)

print(test_generator_indices)

In [ ]:
import seaborn as sns
fig = plt.figure(figsize=(10,10))
sns.heatmap(matrix, annot=True, cmap='viridis')
plt.xticks(ticks=np.arange(4) + 0.4, labels=test_generator.class_indices, rotation=90)
plt.yticks(ticks=np.arange(4) + 0.4, labels=test_generator.class_indices, rotation=0)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
fig.savefig("Confusion Matrix",dpi=700)

In [ ]:
print("Classification Report:\n", report)

In [ ]:
!nvidia-smi

import shutil

def zip_folder(folder_path, zip_filename):
    try:
        shutil.make_archive(zip_filename, 'zip', folder_path)
        print(f"Folder '{folder_path}' successfully zipped to '{zip_filename}.zip'")
    except Exception as e:
        print(f"Error: {e}")

# Example usage
folder_to_zip = '/LAB/Brain Tumor/runs/clas/train8'
zip_file_name = '/LAB/Brain Tumor/Brain Tumor MRI/train8'

zip_folder(folder_to_zip, zip_file_name)
